# Group level statistics


**First you will need to download the example dataset by executing the cell below:**

In [ ]:
print("Downloding the example dataset ... please wait ... ")
![ ! -d "FaceRecognition" ] && wget "FaceRecognition.zip" "https://dl.dropboxusercontent.com/s/q030cu844joczm6/FaceRecognition.zip"
![ ! -d "FaceRecognition" ] && unzip -q ~/hands-on/FaceRecognition.zip -d ~/hands-on && rm ~/hands-on/FaceRecognition.zip 
print("Dataset downloaded, carry on!")

## Retrieving First-Level results
As we now have the same contrast results from multiple `subjects`, we can define our `group level model`. First, we need to gather the `individual contrast maps`.

In [ ]:
import os
from bids.layout import BIDSLayout

ds_path = 'FaceRecognition'
# Initialize the BIDS layout and include the derivatives in it
layout = BIDSLayout(os.path.join(ds_path, 'data/bids'), derivatives = True)
# Attach the results folder to the layout. It must complay with BIDS standards. 
# And must include dataset_description.json file!
layout.add_derivatives(os.path.join(ds_path, "results", "first-level-2mm"))

Let's collect individual t-maps (`stat`) that represent the BOLD activity estimate divided by the uncertainty about this estimate. 

Let's first look at only the **Faces > Scrambled** contrast. 

In [ ]:
contrast = 'FacesScrambled'
stat_files = layout.get(desc = contrast, suffix='stat', extension = '.nii.gz')
print(*stat_files, sep = "\n")

We will want to display subject ID on top of their individual t-maps. Therefore we need to link each stat file with the corresponding subject ID. There are several ways to do this. The simplest seems to retrieve the subject list from the BIDS layout. But `PyBIDS` returns unsorted subject list, that's a bit problematic. And for some reason, sort() does not work on it. 

In [ ]:
print(layout.get_subjects())

Therefore I will sort it this way: 

In [ ]:
subjects = sorted(list(set([f.get_entities().get("subject") for f in stat_files])))
print(subjects)

## Displaying subject t-maps

In [ ]:
from nilearn import plotting
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

for i, stat_map in enumerate(stat_files):
    plotting.plot_glass_brain(stat_map.path, 
                              title = 'sub-' + subjects[i],
                              axes = axes[i],
                              plot_abs = False, 
                              display_mode='x')
fig.suptitle(contrast + ', unthresholded t-maps')
plotting.show()

## Estimate second level model

### Design matrix

The next step includes the definition of a `design matrix`. Here we will want to run a simple `one-sample t-test`. We just need to indicate as many `1` as we have subjects with first-level results.

In [ ]:
import pandas as pd
design_matrix = pd.DataFrame(
    [1] * len(stat_files),
    columns=['intercept'])
design_matrix

### Model specification and fit

In [ ]:
from nilearn.glm.second_level import SecondLevelModel

second_level_model = SecondLevelModel(smoothing_fwhm = 8.0)
second_level_model = second_level_model.fit(
    stat_files,
    design_matrix = design_matrix)

### Contrast estimation

In [ ]:
z_map = second_level_model.compute_contrast(output_type='z_score')

### Thresholding and plotting

In the example dataset, there are only 2 subjects. So, the results will be rather poor. Strict thresholding wouldn't get any significant voxels.

In [ ]:
from nilearn.glm.thresholding import threshold_stats_img

cluster_map, threshold = threshold_stats_img(
    z_map, alpha=0.05, 
    height_control='fpr', 
    cluster_threshold=20,
    two_sided=False)

# displaying on mni152 template brain
from nilearn.datasets import load_mni152_template
template = load_mni152_template()

print('Uncorrected p<.05 threshold: %.3f' % threshold)
plotting.plot_stat_map(
    cluster_map, 
    threshold = threshold,       
    display_mode = 'ortho',
    cut_coords = [37,-62,-14],
    black_bg = True,
    bg_img = template,
    title = contrast + ' ()')

fig = plt.gcf()
fig.set_size_inches(10,3)
plt.show()

In [ ]:
cluster_map, threshold = threshold_stats_img(
    z_map, alpha=.05, 
    height_control='bonferroni', 
    cluster_threshold=0,
    two_sided=False)

print('FWE p<.05 threshold: %.3f' % threshold)
plotting.plot_stat_map(
    cluster_map, 
    threshold = threshold,       
    display_mode = 'ortho',
    cut_coords = [37,-62,-14],
    black_bg = True,
    bg_img = template,
    title = contrast + ' (FWE p<.05)')

fig = plt.gcf()
fig.set_size_inches(10,3)
plt.show()

We can also look at a 3D brain using `plotly`.

In [ ]:
from nilearn import datasets
fsaverage = datasets.fetch_surf_fsaverage()

cluster_map, threshold = threshold_stats_img(
    z_map, alpha = 0.05, 
    height_control = 'fpr',
    cluster_threshold = 20,
    two_sided = True)

view = plotting.view_img_on_surf(cluster_map, threshold=threshold)
# view.open_in_browser()
view

Let's get some more summary results. Here we will get a cluster table. 

In [ ]:
from nilearn.reporting import get_clusters_table
get_clusters_table(z_map, threshold, cluster_threshold=20, two_sided=False, min_distance=8.0)

Can create more complete reports with [nilearn.reporting](https://nilearn.github.io/dev/modules/generated/nilearn.reporting.make_glm_report.html).

In [ ]:
"""
from nilearn.reporting import make_glm_report

report = make_glm_report(model = second_level_model,
                         contrasts = 'intercept',
                         threshold = 3,
                         cluster_threshold = 30,
                         display_mode = 'ortho'
                         )

report
"""

Or we can use ['atlasreader'](https://github.com/miykael/atlasreader) package, as demonstrated in the below script to create reports for all our contrasts. 

## Second level for multiple contrasts

The script below will generate outputs of group-level results for multiple contrasts. 

In this example, we want to threshold the results to **p < .01, uncorrected, with cluster threshold 10**. **(We only have 2 subjects in this example dataset, a more strict threshold might not return any significant results. But you can give it a try!)**

In addition to getting our result maps, we will use *atlasreader* to generate result tables and peak cluster images. 

In [ ]:
import warnings;
warnings.filterwarnings('ignore');

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# ======================================================================
# Dace Apšvalka (MRC CBU 2022)
# Group level fMRI analysis using Nilearn
# ======================================================================

# ======================================================================
# IMPORT RELEVANT PACKAGES
# ======================================================================
import os
import numpy as np
import pandas as pd
from bids.layout import BIDSLayout
from nilearn.datasets import load_mni152_template
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm.thresholding import threshold_stats_img
from nilearn.reporting import get_clusters_table
from atlasreader import create_output


# ======================================================================
# WHICH CONTRASTS
# ======================================================================
contrasts = {'FamousUnfamiliar': 'Famous > Unfamiliar',
             'UnfamiliarFamous': 'Unfamiliar > Famous',
             'FacesScrambled': 'Faces > Scrambled',
             'ScrambledFaces': 'Scrambled > Faces'}
# ======================================================================
# WHAT THRESHOLD
# ======================================================================
height_control ='fpr' # bonferroni is FWE; fpr is uncorrected; 
alpha = .01
cluster_threshold = 10

# ======================================================================
# DEFINE PATHS
# ======================================================================
ds_path = 'FaceRecognition'

outdir = os.path.join(ds_path, 'results', 'group-level_' + height_control + str(alpha)[2:] + 'k' + \
                      str(cluster_threshold))
if not os.path.exists(outdir):
    os.makedirs(outdir)
                      
# ======================================================================
# PREPARE OTHER SUFF
# ======================================================================

# Initialize the BIDS layout and include the derivatives in it
layout = BIDSLayout(os.path.join(ds_path, 'data/bids'), derivatives=True)
layout.add_derivatives(os.path.join(ds_path, "results", "first-level-2mm"))

# load a template to resample images to if needed
template = load_mni152_template()

# ======================================================================
# PERFORM GROUP LEVEL ANALYSIS PER CONTRAST
# ======================================================================

for contrast_id, contrast_val in contrasts.items():
    # get the first level result maps
    stat_files = layout.get(desc = contrast_id, suffix = 'stat', extension = '.nii.gz')
    
    # generate the result file name
    result_name = 'group_zmap_' + contrast_id + '_' + height_control + str(alpha)[2:] + 'k' + str(cluster_threshold)
    
    # create the group level design matrix. Here just a one-sample t-test
    design_matrix = pd.DataFrame([1] * len(stat_files),
                                 columns=['intercept'])
    
    # define and compute the glm
    second_level_model = SecondLevelModel(smoothing_fwhm = 8.0)
    second_level_model = second_level_model.fit(
        stat_files,
        design_matrix = design_matrix)
    
    # get the z-map for the contrast of interest
    z_map = second_level_model.compute_contrast(output_type='z_score')
        
    # threshold the z-map
    thresholded_map, threshold = threshold_stats_img(
        z_map, 
        alpha = alpha,
        height_control = height_control,
        two_sided = False,
        cluster_threshold = cluster_threshold)
    
    # get peaks   
    peaks = get_clusters_table(thresholded_map, stat_threshold=threshold)
    
    # if there is any peak then save the map and generate report
    if len(peaks) != 0:
        print('Creating output for', contrast_val)
        fname = os.path.join(outdir, result_name + '.nii.gz')
        thresholded_map.to_filename(fname)

        # generate and save also atlasreader output
        create_output(
            fname, 
            cluster_extent = cluster_threshold, 
            voxel_thresh = threshold,
            direction = 'pos',
            outdir = os.path.join(outdir, 'atlasreader', contrast_id)
        )
    else:
        print('No significant voxels for', contrast_val)

print('done')

### Summary results

Let's now loop through the generated output and display the peak image and results table for each contrast.

In [ ]:
import glob
from IPython.display import Image

contrasts = {'FamousUnfamiliar': 'Famous > Unfamiliar',
             'UnfamiliarFamous': 'Unfamiliar > Famous',
             'FacesScrambled': 'Faces > Scrambled',
             'ScrambledFaces': 'Scrambled > Faces'}

for contrast_id, contrast_val in contrasts.items():
    # results folder
    results_dir = os.path.join(ds_path, 'results', 'group-level_fpr01k10', 'atlasreader', contrast_id)
    
    # if the directory exists
    if os.path.exists(results_dir):
        print('Results for', contrast_val)
        
        # find the image file        
        cluster1 = glob.glob(os.path.join(results_dir, '*cluster01.png'))        
        # find peak table
        peaks = glob.glob(os.path.join(results_dir, '*_peaks.csv'))
        
        # display the image
        display(Image(cluster1[0]))
        # display the table
        display(pd.read_csv(peaks[0]))
    else:
        print(contrast_val, 'does not have significant results')            